In [1]:
import pandas as pd
from datetime import datetime
import sqlite3 as sql

In [7]:
conn = sql.connect('adventure works new.db')

In [9]:
def run_query(query): return pd.read_sql_query(query,conn)

In [17]:
customer_query = """ SELECT CustomerKey, BirthDate FROM AdventureWorks_Customers; """
customers_df = run_query(customer_query)

In [19]:
customers_df['BirthDate'] = pd.to_datetime(customers_df['BirthDate'])

In [23]:
current_year = datetime.now().year
customers_df['Age'] = current_year - customers_df [ 'BirthDate'].dt.year

In [25]:
current_date = datetime.now()
customers_df['Age']-= ((customers_df['BirthDate'].dt.month > current_date.month) |
                       ((customers_df['BirthDate'].dt.month == current_date.month) &
                        (customers_df['BirthDate'].dt.day > current_date.day)))


In [27]:
age_bins =[0, 17, 24, 34, 44, 54, 64, 100]
age_labels = ['Under 18', '18-24', '25-34', '35-44', '45-54', '55-64', '65+' ]

customers_df ['Age_Group'] = pd.cut(customers_df['Age'], bins=age_bins, labels=age_labels, right=True)

In [29]:
conn= sql.connect('adventure works new.db')

In [31]:
alter_query = """ ALTER TABLE AdventureWorks_Customers ADD COLUMN Age INTEGER;"""

In [33]:
conn.execute(alter_query)

In [35]:
alter_query = """ ALTER TABLE AdventureWorks_Customers ADD COLUMN Age_Group TEXT ; """

In [37]:
conn.execute(alter_query)

In [39]:
conn.commit()

In [48]:
for index, row in customers_df.iterrows():
    update_query = """
    UPDATE AdventureWorks_Customers 
    SET Age = ? ,
        Age_Group=?
    WHERE CustomerKey = ?
    """
    conn.execute(update_query, (row['Age'], row['Age_Group'], row['CustomerKey']))

In [50]:
conn.commit()

In [52]:
conn.close()

In [62]:
conn = sql.connect('adventure works new.db')
updated_df = pd.read_sql_query("SELECT CustomerKey, BirthDate, Age, Age_Group FROM AdventureWorks_Customers", conn)
print(updated_df.head())                              

   CustomerKey  BirthDate  Age Age_Group
0        11000   4/8/1966   58     55-64
1        11001  5/14/1965   59     55-64
2        11002  8/12/1965   59     55-64
3        11003  2/15/1968   56     55-64
4        11004   8/8/1968   56     55-64


In [64]:
conn.close()